In [6]:
# pull
%cd "/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images"
!git pull

/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images
Already up to date.


In [ ]:
# commit
!git config --global user.email "ssharon95@gmail.com"
!git config --global user.name "SharonPeled"
!git commit -a -m "handling cuda out of memory error"

In [ ]:
# push
!git config --global user.email "ssharon95@gmail.com"
!git config --global user.name "SharonPeled"
%cd "/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images"
!git push

In [7]:
!cat "/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/CheXpert-baseline/utils.py"

from dataclasses import dataclass
import numpy as np
import os
import torch
from torch import nn
from torchmetrics.functional import auc
import random
import datetime


@dataclass
class Configs:
    SEED = 123
    NUM_CLASSES = 5
    ANNOTATIONS_COLUMNS = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Pleural Effusion"]
    UONES_COLUMNS = ["Edema", "Pleural Effusion", "Atelectasis"]
    UZEROS_COLUMNS = ["Cardiomegaly", "Consolidation"]


def set_seed():
    torch.manual_seed(Configs.SEED)
    random.seed(Configs.SEED)
    np.random.seed(Configs.SEED)


def to_gpu(x):
    return x.cuda() if torch.cuda.is_available() else x


def get_time_str():
    time_str = str(datetime.datetime.now())[:-10]
    trans = str.maketrans("-: ","__-")
    return time_str.translate(trans)


def create_checkpoint(model, epoch, i, valid_dataloader, criterion, results, TrainingConfigs):
    valid_loss, valid_auc = calc_auc_score(model, valid_dataloader, criterion)
    results['valid_loss'].append

In [15]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=3798e41af0dc7f42515bcae51bb88a317f75c50cfb1e2813236694854b8c89a0
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built gputil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 12.0 GB  |     Proc size: 749.9 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [1]:
import psutil
import humanize
import os
import GPUtil as GPU
import torch 

torch.cuda.empty_cache()
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.6 GB  |     Proc size: 344.0 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total     15109MB


In [ ]:
!pkill -9 -f ipykernel_launcher

In [14]:
import os 
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/CheXpert-baseline'

In [5]:
%cd "/home/"
%ls

/home


In [6]:
!wget "https://eur01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fus13.mailchimp.com%2Fmctx%2Fclicks%3Furl%3Dhttp%253A%252F%252Fdownload.cs.stanford.edu%252Fdeep%252FCheXpert-v1.0-small.zip%26h%3Dd560d6d552da464fcc07b567091b41f6024e92218c1f46c47d0c2750d094e9a5%26v%3D1%26xid%3D69567474fa%26uid%3D55365305%26pool%3Dcontact_facing%26subject%3DCheXpert-v1.0%253A%2BSubscription%2BConfirmed&data=05%7C01%7Csharonpe%40campus.technion.ac.il%7Cde77057e56b8487a125408da492d649b%7Cf1502c4cee2e411c9715c855f6753b84%7C1%7C0%7C637902757328007032%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&sdata=jRoszYt%2F8%2B%2Fe5PlGojS4CCWb%2F2R1l9e32TX5weyYpX8%3D&reserved=0"

The name is too long, 595 chars total.
Trying to shorten...
New name is index.html?url=https:%2F%2Fus13.mailchimp.com%2Fmctx%2Fclicks?url=http%3A%2F%2Fdownload.cs.stanford.edu%2Fdeep%2FCheXpert-v1.0-small.zip&h=d560d6d552da464fcc07b567091b41f6024e92218c1f46c47d0c2750d094e9a5&v=1&xid=69567474fa&uid=55365305&.
--2022-06-16 16:24:49--  https://eur01.safelinks.protection.outlook.com/?url=https%3A%2F%2Fus13.mailchimp.com%2Fmctx%2Fclicks%3Furl%3Dhttp%253A%252F%252Fdownload.cs.stanford.edu%252Fdeep%252FCheXpert-v1.0-small.zip%26h%3Dd560d6d552da464fcc07b567091b41f6024e92218c1f46c47d0c2750d094e9a5%26v%3D1%26xid%3D69567474fa%26uid%3D55365305%26pool%3Dcontact_facing%26subject%3DCheXpert-v1.0%253A%2BSubscription%2BConfirmed&data=05%7C01%7Csharonpe%40campus.technion.ac.il%7Cde77057e56b8487a125408da492d649b%7Cf1502c4cee2e411c9715c855f6753b84%7C1%7C0%7C637902757328007032%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&sdata=jRoszYt%2F8%2B%2Fe

In [7]:
prev = 'index.html?url=https:%2F%2Fus13.mailchimp.com%2Fmctx%2Fclicks?url=http%3A%2F%2Fdownload.cs.stanford.edu%2Fdeep%2FCheXpert-v1.0-small.zip&h=d560d6d552da464fcc07b567091b41f6024e92218c1f46c47d0c2750d094e9a5&v=1&xid=69567474fa&uid=55365305&'
!mv "index.html?url=https:%2F%2Fus13.mailchimp.com%2Fmctx%2Fclicks?url=http%3A%2F%2Fdownload.cs.stanford.edu%2Fdeep%2FCheXpert-v1.0-small.zip&h=d560d6d552da464fcc07b567091b41f6024e92218c1f46c47d0c2750d094e9a5&v=1&xid=69567474fa&uid=55365305&" "CheXpert-v1.0-small.zip"


In [8]:
import zipfile
from tqdm.notebook import tqdm
path_to_zip_file = r"/home/CheXpert-v1.0-small.zip"
directory_to_extract_to = r"/home"
with zipfile.ZipFile(path_to_zip_file) as zf:
  for member in tqdm(zf.infolist(), desc='Extracting '):
    try:
      zf.extract(member, directory_to_extract_to)
    except zipfile.error as e:
      pass

Extracting :   0%|          | 0/476235 [00:00<?, ?it/s]

In [25]:
%load_ext autoreload
%autoreload 2

In [10]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/CheXpert-baseline"
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/CheXpert-baseline")
# !pip install -r "/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/requirements.txt"
!pip install torchmetrics

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/debiasing-racial-effect-in-medical-images/CheXpert-baseline
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 37.5 MB/s 


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass

# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_rows', 500)
# import warnings
# warnings.filterwarnings('ignore')
# C:/Users/sshar/AppData/Roaming/jupyter/nbextensions/snippets /snippets.json (jupyter --data-dir)

In [27]:
from dataset import CheXpertDataset
import utils
from utils import to_gpu

In [2]:
@dataclass
class TrainingConfigs:
    BATCH_SIZE = 2
    DATA_DIR = "/home"
    EPOCHS = 3
    LEARNING_RATE = 0.0001
    CHECKPOINT_TIME_INTERVAL = 3 # seconds
    CHECKPOINT_DIR = r"model_checkpoints"
    MODEL_VERSION = "densenet121"
    TRAINED_MODEL_PATH = None
    TRAIN_LOADER_SIZE = None
    VALID_LOADER_SIZE = None

NameError: ignored

In [18]:
utils.set_seed()

In [19]:
train_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [20]:
# Create data loaders.
train_dataset = CheXpertDataset(mode='train', data_dir=TrainingConfigs.DATA_DIR, transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=True)
TrainingConfigs.TRAIN_LOADER_SIZE = len(train_dataloader)
len(train_dataset)

223414

In [21]:
valid_dataset = CheXpertDataset(mode='valid', data_dir=TrainingConfigs.DATA_DIR, transform=valid_transform)
valid_dataset.labels = valid_dataset.labels # hack for speed debugging
valid_dataloader = DataLoader(valid_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.VALID_LOADER_SIZE = len(valid_dataloader)
len(valid_dataset)

234

In [22]:
torch.hub._validate_not_a_forked_repo = lambda a,b,c: True # workaround for torch.hub
model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
model.classifier = nn.Linear(in_features=1024, out_features=utils.Configs.NUM_CLASSES, bias=True) # updating model output dim

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip


In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=TrainingConfigs.LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [30]:
# TODO: try except in create_checkpoint to save model anyway
# TODO: move print to create_checkpoint
# TODO: update training to handle any batch size
# TODO: when cuda fails to reload kernel
model, results, last_epoch, last_iter = utils.get_previos_training_place(model, TrainingConfigs)
last_epoch, last_iter
model.train()
model = to_gpu(model)
start_time = time.time()
try:
  for epoch in range(last_epoch, TrainingConfigs.EPOCHS):
    train_dataloader_iter = islice(tqdm(enumerate(train_dataloader), total=len(train_dataloader)), 
                                    last_iter+1, len(train_dataloader)) # fast foward dataloader
    for i, (images, labels) in train_dataloader_iter:
        images = to_gpu(images)
        labels = to_gpu(labels)        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        results['train_loss'].append(loss.item())
        if time.time()-start_time > TrainingConfigs.CHECKPOINT_TIME_INTERVAL:
            utils.create_checkpoint(model, epoch, i, valid_dataloader, criterion, results, TrainingConfigs)
            start_time = time.time()
except Exception as e:
  utils.create_checkpoint(model, epoch, i, valid_dataloader, criterion, results, TrainingConfigs)
  print(e)

  0%|          | 0/111707 [00:00<?, ?it/s]

RuntimeError: ignored